# SOA - Ejemplos MPI

Ejemplos utilizados en la materia SOA utilizando MPI, codificados con el  lenguaje C. Compilados con el compilado MPI.  Ejecutado en el entorno Colab. 


---
## 1.Ejemplo de MPI (Hola Mundo)

In [ ]:
code = """

#include <string.h>
#include <stdio.h>
#include <mpi.h>

int main(int argc, char** argv)
{
  int myRank, numProcs;
  int tag=1;
  char mensaje[100];
  MPI_Status status;

  MPI_Init(NULL, NULL);

  MPI_Comm_rank( MPI_COMM_WORLD, &myRank );
  MPI_Comm_size( MPI_COMM_WORLD, &numProcs );

  if( myRank != 0 )
  {
	  int dest = 0;
    sprintf( mensaje, "Hola mundo, soy el proceso %d de %d", myRank, numProcs );	
 
    MPI_Send(&mensaje, strlen(mensaje)+1, MPI_CHAR, dest, tag, MPI_COMM_WORLD);
  }
  else
  {
    for(int i=1; i<numProcs; i++)
    {
      MPI_Recv(&mensaje, 100, MPI_CHAR, i, tag, MPI_COMM_WORLD, &status);
      printf( "%s\\n", mensaje );
    }
  }

  MPI_Finalize();
}
"""
text_file = open("hello.c", "w")
text_file.write(code)
text_file.close()

## 1.1.Compilación de código C Hola Mundo. 

In [ ]:
!mpicc -o hello hello.c

## 1.2.Ejecución Hola Mundo en MPI.

In [ ]:
!mpirun --allow-run-as-root -oversubscribe -n 30 hello

---
# 2.Ejemplo MPI_Bcast


## **2.1 En C**

In [ ]:
code= """
#include <mpi.h>
#include <stdio.h>

int main(int argc, char** argv) {
        int rank;
        int buf;
        const int root=0;

        MPI_Init(&argc, &argv);
        MPI_Comm_rank(MPI_COMM_WORLD, &rank);

        if(rank == root) {
           buf = 777;
        }

        printf("[%d]: Before Bcast, buf is %d\\n", rank, buf);

        // common region between all nodes
        MPI_Bcast(&buf, 1, MPI_INT, root, MPI_COMM_WORLD);

        printf("[%d]: After Bcast, buf is %d\\n", rank, buf);

        MPI_Finalize();
        return 0;
}
"""
text_file = open("bcast.c", "w")
text_file.write(code)
text_file.close()

In [ ]:
!mpicc -o bcast bcast.c

In [ ]:
!mpirun --allow-run-as-root -oversubscribe  -np 4 bcast

## **2.2 En Python**

In [ ]:
!python -m pip install mpi4py

In [ ]:
%%writefile bcast.py
from mpi4py import MPI

comm = MPI.COMM_WORLD
size = comm.Get_rank()
rank = comm.Get_rank()

root = 0
buf = 0
if rank == root:
    buf = 777

print ("[",rank,"]: Before Bcast, buf is",buf)
buf = comm.bcast(buf, root=root)
print ("[",rank,"]: after Bcast, buf is",buf)

In [ ]:
! mpirun --allow-run-as-root -oversubscribe -np $4 python bcast.py